Efficiency Example
====

The first cell is just helper functions; the second cell is where everything happens

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple
from fourbody.param import helicity_param

In [ ]:
def _read_branches(tree, i: int) -> np.ndarray:
    """
    Read momentum and energy branches for the i'th pion
    
    """
    return np.row_stack((tree[f"Pion{i}_TRUEP_{x}"] for x in ("X", "Y", "Z", "E")))


def _read_mc() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Read that MC file you gave me
    
    Returns arrays: (px, py, pz, E)
    
    :returns: (4, N) shape numpy array for pi1
    :returns: (4, N) shape numpy array for pi2
    :returns: (4, N) shape numpy array for pi3
    :returns: (4, N) shape numpy array for pi4
    
    """
    with uproot.open("Total_DK_as_DK_MC_2017_After_PIC_Calib2_pols_merged_with_PID.root") as f:
        tree = f["DecayTree"]
        
        pi1, pi2, pi3, pi4 = (_read_branches(tree, i) for i in range(1, 5))
        
    return pi1, pi2, pi3, pi4
        

In [ ]:
# Read the right branches in the data file - we just care about kinematic information (momenta, energy)
mc_pi1, mc_pi2, mc_pi3, mc_pi4 = _read_mc()

# Generate "flat" (phase space distributed) events to compare the MC to
flat_pi1, flat_pi2, flat_pi3, flat_pi4 = _generate_flat()

# Parameterise both into 5d point
# NB this function accepts (pi+ pi+ pi- pi-) (or pi- pi- pi+ pi+)
mc = helicity_param(mc_pi1, mc_pi3, mc_pi2, mc_pi4)
flat = helicity_param(flat_pi1, flat_pi2, flat_pi3, flat_pi4)

# Train a reweighter

# Plot